In [1]:
pip install diffusers transformers torch numpy Pillow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/torch_stable.html
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
from diffusers import StableDiffusionPipeline, ControlNetModel
from PIL import Image
import numpy as np
import torch
import time
import cv2

In [3]:
!git clone https://github.com/CompVis/stable-diffusion

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 340 (delta 109), reused 92 (delta 92), pack-reused 182 (from 1)
Receiving objects: 100% (340/340), 42.58 MiB | 2.42 MiB/s, done.
Resolving deltas: 100% (131/131), done.


In [4]:
!git clone https://github.com/lllyasviel/ControlNet

Cloning into 'ControlNet'...
remote: Enumerating objects: 1356, done.
remote: Total 1356 (delta 0), reused 0 (delta 0), pack-reused 1356 (from 1)
Receiving objects: 100% (1356/1356), 122.40 MiB | 3.16 MiB/s, done.
Resolving deltas: 100% (599/599), done.


### Checkpoint Setup
#### Download the checkpoint files for Stable Diffusion and ControlNet from Hugging Face, which are required to generate images based on text prompts and depth maps.

In [2]:
from diffusers import StableDiffusionPipeline, ControlNetModel

In [3]:
pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

diffusion_pytorch_model.safetensors:   5%|4         | 168M/3.44G [00:00<?, ?B/s]

model.safetensors:  49%|####8     | 241M/492M [00:00<?, ?B/s]

model.safetensors:  22%|##2       | 273M/1.22G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:  69%|######8   | 231M/335M [00:00<?, ?B/s]

model.safetensors:  28%|##8       | 346M/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   6%|6         | 220M/3.44G [00:00<?, ?B/s]

model.safetensors:  62%|######1   | 304M/492M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-depth").to("mps")


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [14]:
prompt = "A beautiful mountain landscape with clear skies"
